In [2]:
import time
import requests
import re
from difflib import get_close_matches
from math import floor
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer
from codecarbon import EmissionsTracker

# --------------------
# CONFIGURATION
# --------------------
# --- Model Identifiers ---
# Switched from 'facebook/bart-large-mnli' to the new Phi-3 model.
CLOUD_MODEL_ID = "microsoft/Phi-3-medium-128k-instruct"
LOCAL_MODEL_ID = "microsoft/phi-3-mini-4k-instruct" # The model loaded in your LM Studio

# --- API Endpoints and Authentication ---
# Updated API URL to point to the new model's inference endpoint.
HF_API_URL = f"https://api-inference.huggingface.co/models/{CLOUD_MODEL_ID}"
HF_TOKEN = "xxxxxxxxxxxxxxxxxxxxxx" # Your Hugging Face API key
HF_HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"}

LM_API_URL = "http://127.0.0.1:1234/v1/chat/completions"  # LM Studio local API

# --- Experiment Settings ---
RANDOM_SEED = 42  # Use None for a different random sample each run

# --- Tokenizer ---
# Using the tokenizer for the new cloud model for more accurate token counts.
# This ensures consistency in token estimation for both models.
tokenizer = AutoTokenizer.from_pretrained(CLOUD_MODEL_ID)

# --------------------
# HELPER FUNCTIONS
# --------------------
def estimate_tokens(*texts):
    """Estimate token count for given strings using the model's tokenizer."""
    total_tokens = 0
    for txt in texts:
        if txt:
            # We set add_special_tokens to False to count only the content tokens.
            total_tokens += len(tokenizer.encode(txt, add_special_tokens=False))
    return total_tokens

def robust_map_label(pred_text, labels):
    """Map model output text to one of the known labels using fuzzy matching."""
    if not pred_text:
        return labels[0] # Return a default label if the prediction is empty.
    # Clean the predicted text to remove non-alphabetic characters.
    clean_pred = re.sub(r'[^A-Za-z ]', '', pred_text).strip().lower()
    # Find the best match from the list of official labels.
    match = get_close_matches(clean_pred, [l.lower() for l in labels], n=1, cutoff=0.4)
    if match:
        # Return the correctly cased label.
        return labels[[l.lower() for l in labels].index(match[0])]
    # Fallback: if no good match, check for any token overlap.
    pred_tokens = set(clean_pred.split())
    for label in labels:
        if set(label.lower().split()) & pred_tokens:
            return label
    # If all else fails, return the first label as a fallback.
    return labels[0]

def build_classification_prompt(description, labels):
    """Creates a standardized prompt for the classification task."""
    labels_str = ", ".join(labels)
    # This prompt structure guides the model to perform the classification task.
    return (
        f"<|user|>\n"
        f"Classify the following movie description into ONE of these exact genres: {labels_str}.\n"
        f"Respond with only the single genre name and nothing else.\n\n"
        f"Description: \"{description}\"<|end|>\n"
        f"<|assistant|>"
    )

# --------------------
# MODEL CLASSIFIERS
# --------------------
def classify_cloud(description, labels):
    """
    Classifies a description using the cloud-hosted Phi-3-medium model.
    This function was REWRITTEN to handle an instruction-tuned model instead of a
    dedicated classification model.
    """
    prompt = build_classification_prompt(description, labels)
    # The payload is now for a text-generation task.
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 10, # Limit the response length.
            "return_full_text": False # We only want the generated part.
        }
    }
    start_time = time.time()
    try:
        r = requests.post(HF_API_URL, headers=HF_HEADERS, json=payload)
        r.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        latency = time.time() - start_time
        data = r.json()
        raw_pred = data[0]["generated_text"].strip()
        pred = robust_map_label(raw_pred, labels)
    except requests.exceptions.RequestException as e:
        print(f"Cloud API error: {e}")
        pred, latency = labels[0], time.time() - start_time
    except (KeyError, IndexError) as e:
        print(f"Cloud API response format error: {e} - Response: {r.text}")
        pred, latency = labels[0], time.time() - start_time

    tokens_used = estimate_tokens(prompt, pred)
    return pred, latency, tokens_used

def classify_local(description, labels):
    """Classifies a description using the locally-hosted Phi-3-mini model."""
    prompt = build_classification_prompt(description, labels)
    # The payload for LM Studio's OpenAI-compatible server.
    data = {
        "model": LOCAL_MODEL_ID,
        "messages": [
            # The prompt is simplified as the build function now includes roles.
            {"role": "user", "content": prompt}
        ],
        "temperature": 0,
        "max_tokens": 10,
        "stream": False
    }
    start_time = time.time()
    try:
        r = requests.post(LM_API_URL, json=data)
        r.raise_for_status()
        latency = time.time() - start_time
        resp = r.json()
        raw_pred = resp["choices"][0]["message"]["content"].strip()
        mapped = robust_map_label(raw_pred, labels)
    except requests.exceptions.RequestException as e:
        print(f"Local API error: {e}")
        mapped, latency = labels[0], time.time() - start_time
    except (KeyError, IndexError) as e:
        print(f"Local API response format error: {e} - Response: {r.text}")
        mapped, latency = labels[0], time.time() - start_time

    tokens_used = estimate_tokens(prompt, mapped)
    return mapped, latency, tokens_used

# --------------------
# EVALUATION
# --------------------
def evaluate(results, model_name, carbon_kg=None):
    """Calculates and prints performance metrics."""
    if not results:
        print(f"\n--- {model_name} ---")
        print("No results to evaluate.")
        return

    true = [r['true'] for r in results]
    pred = [r['pred'] for r in results]
    acc = accuracy_score(true, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(true, pred,
                                                      average='weighted',
                                                      zero_division=0)
    avg_lat = sum(r['latency'] for r in results) / len(results)
    avg_tokens = sum(r['tokens'] for r in results) / len(results)
    total_tokens = sum(r['tokens'] for r in results)

    print(f"\n--- {model_name} ---")
    print(f"Samples evaluated: {len(results)}")
    print(f"Accuracy:   {acc:.3f}")
    print(f"Precision:  {prec:.3f}")
    print(f"Recall:     {rec:.3f}")
    print(f"F1 Score:   {f1:.3f}")
    print(f"Avg Latency: {avg_lat:.2f}s")
    print(f"Avg Tokens: {avg_tokens:.1f} | Total Tokens: {total_tokens}")
    if carbon_kg is not None:
        print(f"Estimated CO2 Emission: {carbon_kg:.6f} kg")

# --------------------
# MAIN EXECUTION
# --------------------
if __name__ == "__main__":
    print("Loading IMDb genres dataset...")
    dataset = load_dataset("jquigl/imdb-genres")
    labels = sorted(list(set(dataset["train"]["genre"])))
    print(f"Candidate Genres: {labels}")

    # --- Sample the dataset ---
    test_len = len(dataset["test"])
    num_samples = floor(test_len * 0.10)
    sample_data = dataset["test"].shuffle(seed=RANDOM_SEED).select(range(num_samples))
    print(f"Sampling {num_samples} rows out of {test_len} test samples ({100*num_samples/test_len:.1f}%).")

    # --- Separate carbon tracking for each model ---
    cloud_tracker = EmissionsTracker(project_name="Cloud Model (Phi-3 Medium)")
    local_tracker = EmissionsTracker(project_name="Local Model (Phi-3 Mini)")

    # --- Cloud model run ---
    cloud_results = []
    print(f"\n>>> Running Cloud Model ({CLOUD_MODEL_ID})...")
    cloud_tracker.start()
    for item in sample_data:
        desc, gold = item["description"], item["genre"]
        pred, lat, tokens = classify_cloud(desc, labels)
        cloud_results.append({"true": gold, "pred": pred, "latency": lat, "tokens": tokens})
        print(f"[Cloud] True: {gold:<12} | Pred: {pred:<12} | Lat: {lat:.2f}s | Tokens: {tokens}")
    cloud_emissions = cloud_tracker.stop()

    # --- Local model run ---
    local_results = []
    print(f"\n>>> Running Local Model ({LOCAL_MODEL_ID})...")
    local_tracker.start()
    for item in sample_data:
        desc, gold = item["description"], item["genre"]
        pred, lat, tokens = classify_local(desc, labels)
        local_results.append({"true": gold, "pred": pred, "latency": lat, "tokens": tokens})
        print(f"[Local] True: {gold:<12} | Pred: {pred:<12} | Lat: {lat:.2f}s | Tokens: {tokens}")
    local_emissions = local_tracker.stop()

    # --- Final Summary ---
    print("\n" + "="*50)
    print("               FINAL RESULTS SUMMARY")
    print("="*50)

    if cloud_emissions is not None:
        print(f"\nEstimated CO2 emission for Cloud Model run: {cloud_emissions:.6f} kg")
    if local_emissions is not None:
        print(f"Estimated CO2 emission for Local Model run: {local_emissions:.6f} kg")

    # --- Evaluate and print final metrics ---
    evaluate(cloud_results, f"Cloud Model ({CLOUD_MODEL_ID})", cloud_emissions)
    evaluate(local_results, f"Local Model ({LOCAL_MODEL_ID})", local_emissions)


tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\rodri\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rodri\.cache\huggingface\hub\models--microsoft--Phi-3-medium-128k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

[Local] True: Crime        | Pred: Action       | Lat: 0.18s | Tokens: 119
[Local] True: Biography    | Pred: Action       | Lat: 0.19s | Tokens: 129
[Local] True: Romance      | Pred: Romance      | Lat: 0.19s | Tokens: 123
[Local] True: Biography    | Pred: Action       | Lat: 0.18s | Tokens: 109
[Local] True: Adventure    | Pred: Fantasy      | Lat: 0.19s | Tokens: 144
[Local] True: Family       | Pred: Romance      | Lat: 0.13s | Tokens: 137
[Local] True: Biography    | Pred: Biography    | Lat: 0.19s | Tokens: 121
[Local] True: Biography    | Pred: Action       | Lat: 0.19s | Tokens: 141
[Local] True: Romance      | Pred: Action       | Lat: 0.18s | Tokens: 113
[Local] True: Animation    | Pred: Animation    | Lat: 0.18s | Tokens: 135
[Local] True: Animation    | Pred: Fantasy      | Lat: 0.18s | Tokens: 127
[Local] True: Action       | Pred: Adventure    | Lat: 0.18s | Tokens: 135
[Local] True: Thriller     | Pred: Mystery      | Lat: 0.18s | Tokens: 131
[Local] True: Fantasy    

[codecarbon INFO @ 08:55:45] Energy consumed for RAM : 0.001429 kWh. RAM Power : 10.0 W


[Local] True: Thriller     | Pred: Thriller     | Lat: 0.17s | Tokens: 111
[Local] True: Crime        | Pred: Action       | Lat: 0.17s | Tokens: 105


[codecarbon INFO @ 08:55:45] Delta energy consumed for CPU with constant : 0.000547 kWh, power : 432.0 W
[codecarbon INFO @ 08:55:45] Energy consumed for All CPU : 0.061765 kWh
[codecarbon INFO @ 08:55:45] Energy consumed for all GPUs : 0.008298 kWh. Total GPU Power : 61.67133425764411 W
[codecarbon INFO @ 08:55:45] 0.071492 kWh of electricity used since the beginning.



               FINAL RESULTS SUMMARY

Estimated CO2 emission for Cloud Model run: 0.028046 kg
Estimated CO2 emission for Local Model run: 0.020790 kg

--- Cloud Model (microsoft/Phi-3-medium-128k-instruct) ---
Samples evaluated: 2975
Accuracy:   0.128
Precision:  0.016
Recall:     0.128
F1 Score:   0.029
Avg Latency: 0.26s
Avg Tokens: 123.4 | Total Tokens: 367191
Estimated CO2 Emission: 0.028046 kg

--- Local Model (microsoft/phi-3-mini-4k-instruct) ---
Samples evaluated: 2975
Accuracy:   0.242
Precision:  0.336
Recall:     0.242
F1 Score:   0.224
Avg Latency: 0.17s
Avg Tokens: 124.0 | Total Tokens: 368824
Estimated CO2 Emission: 0.020790 kg
